In [2]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/코랩파일들/파이널프로젝트/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

Mounted at /content/drive


'cpu'

In [3]:
import torch.nn as nn

In [104]:
import pandas_datareader.data as web
import datetime as dt
import yfinance as yfin

yfin.pdr_override()
start = dt.datetime(2016,7,8)
end = dt.datetime(2020,5,11)

train = web.get_data_yahoo('^KS11',start = start,end = end)

[*********************100%%**********************]  1 of 1 completed


In [105]:
tmp = web.get_data_yahoo('DX-Y.NYB',start = start,end = end) # 달러인덱스

[*********************100%%**********************]  1 of 1 completed


In [106]:
tmp2 = web.get_data_yahoo('^IXIC',start = start,end = end) # 나스닥

[*********************100%%**********************]  1 of 1 completed


In [107]:
tmp3 = web.get_data_yahoo('^GSPC',start = start,end = end) # S&P500

[*********************100%%**********************]  1 of 1 completed


In [108]:
# train = train.merge(tmp,on = 'Date', how='left',suffixes=('_x', '_y')).dropna(axis=0)
train = train.merge(tmp2,on = 'Date', how='left').dropna(axis=0)
train = train.merge(tmp3,on = 'Date', how='left',suffixes=('_e', '_f')).dropna(axis=0)
train

,Open_x,High_x,Low_x,Close_x,Adj Close_x,Volume_x,Open_y,High_y,Low_y,Close_y,Adj Close_y,Volume_y,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,,,,,,,,,,,,
2016-07-08,1972.280029,1979.750000,1956.910034,1963.099976,1963.099976,356500,4906.660156,4959.000000,4901.270020,4956.759766,4956.759766,1.947260e+09,2106.969971,2131.709961,2106.969971,2129.899902,2129.899902,3607500000
2016-07-11,1979.359985,1990.949951,1977.810059,1988.540039,1988.540039,337800,4976.540039,5002.500000,4976.540039,4988.640137,4988.640137,1.718450e+09,2131.719971,2143.159912,2131.719971,2137.159912,2137.159912,3253340000
2016-07-12,1991.130005,1998.030029,1986.449951,1991.229980,1991.229980,432700,5017.990234,5032.100098,5009.680176,5022.819824,5022.819824,1.866280e+09,2139.500000,2155.399902,2139.500000,2152.139893,2152.139893,4097820000
2016-07-13,2007.650024,2013.599976,1995.839966,2005.550049,2005.550049,357600,5036.319824,5036.379883,5002.819824,5005.729980,5005.729980,1.650820e+09,2153.810059,2156.449951,2146.209961,2152.429932,2152.429932,3502320000
2016-07-14,2004.119995,2009.670044,1997.939941,2008.770020,2008.770020,340200,5041.950195,5045.180176,5025.149902,5034.060059,5034.060059,1.641170e+09,2157.879883,2168.989990,2157.879883,2163.750000,2163.750000,3465610000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-29,1936.890015,1957.510010,1934.310059,1947.560059,1947.560059,1984200,8802.700195,8957.269531,8765.009766,8914.709961,8914.709961,4.407140e+09,2918.459961,2954.860107,2912.159912,2939.510010,2939.510010,6645640000
2020-05-04,1906.420044,1918.660034,1894.290039,1895.369995,1895.369995,1146900,8555.320312,8715.820312,8537.830078,8710.709961,8710.709961,3.443140e+09,2815.010010,2844.239990,2797.850098,2842.739990,2842.739990,4735930000
2020-05-06,1919.449951,1928.760010,1919.449951,1928.760010,1928.760010,1082200,8874.700195,8933.250000,8819.370117,8854.389648,8854.389648,3.656750e+09,2883.139893,2891.110107,2847.649902,2848.419922,2848.419922,4892570000


In [109]:
mins = train.min(axis=0)
sizes = train.max(axis=0)- mins
train_ft = (train-mins)/sizes
train_ft

,Open_x,High_x,Low_x,Close_x,Adj Close_x,Volume_x,Open_y,High_y,Low_y,Close_y,Adj Close_y,Volume_y,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,,,,,,,,,,,,
2016-07-08,0.446100,0.424634,0.450719,0.443172,0.443172,0.095722,0.000000,0.000000,0.000000,0.000000,0.000000,0.259050,0.017877,0.025215,0.017899,0.034374,0.034374,0.297904
2016-07-11,0.452445,0.434906,0.468923,0.465477,0.465477,0.085333,0.014283,0.008915,0.015437,0.006559,0.006559,0.208784,0.036964,0.034061,0.037010,0.039955,0.039955,0.252250
2016-07-12,0.462992,0.441400,0.476448,0.467836,0.467836,0.138056,0.022755,0.014981,0.022234,0.013591,0.013591,0.241260,0.042964,0.043516,0.043018,0.051469,0.051469,0.361111
2016-07-13,0.477795,0.455679,0.484627,0.480391,0.480391,0.096333,0.026502,0.015859,0.020827,0.010075,0.010075,0.193926,0.054000,0.044328,0.048199,0.051692,0.051692,0.284346
2016-07-14,0.474632,0.452075,0.486456,0.483214,0.483214,0.086667,0.027652,0.017662,0.025407,0.015904,0.015904,0.191806,0.057139,0.054015,0.057210,0.060393,0.060393,0.279614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-29,0.414388,0.404237,0.431035,0.429547,0.429547,1.000000,0.796323,0.819423,0.792427,0.814323,0.814323,0.799453,0.643708,0.661123,0.639648,0.656687,0.656687,0.689548
2020-05-04,0.387084,0.368606,0.396178,0.383789,0.383789,0.534833,0.745760,0.769940,0.745834,0.772351,0.772351,0.587675,0.563926,0.575665,0.551381,0.582304,0.582304,0.443369
2020-05-06,0.398760,0.377869,0.418092,0.413064,0.413064,0.498889,0.811039,0.814501,0.803576,0.801912,0.801912,0.634602,0.616468,0.611874,0.589835,0.586670,0.586670,0.463561


In [110]:
train_ft.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 908 entries, 2016-07-08 to 2020-05-08
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Open_x       908 non-null    float64
 1   High_x       908 non-null    float64
 2   Low_x        908 non-null    float64
 3   Close_x      908 non-null    float64
 4   Adj Close_x  908 non-null    float64
 5   Volume_x     908 non-null    float64
 6   Open_y       908 non-null    float64
 7   High_y       908 non-null    float64
 8   Low_y        908 non-null    float64
 9   Close_y      908 non-null    float64
 10  Adj Close_y  908 non-null    float64
 11  Volume_y     908 non-null    float64
 12  Open         908 non-null    float64
 13  High         908 non-null    float64
 14  Low          908 non-null    float64
 15  Close        908 non-null    float64
 16  Adj Close    908 non-null    float64
 17  Volume       908 non-null    float64
dtypes: float64(18)
memory usage: 13

In [111]:
def transform_data(data,mins,sizes,seq_len=60,pred_len=1):

    # data = (data-mins)/sizes

    col_len = data.shape[-1]
    window_size = seq_len+pred_len
    data_len = len(data)-window_size+1 # 0부터 시작하니까
    new_arr = data[:data_len]
    # print(new_arr)
    for i in range(1,window_size):

        data = np.delete(data,0,axis=0)
        new_arr = np.concatenate([new_arr,data[:data_len]],axis=1)
    new_arr = new_arr.reshape(data_len,window_size,col_len)

    x_arr = new_arr[:,:seq_len]
    y_arr = new_arr[:,seq_len:,3]

    return x_arr,y_arr,new_arr

In [112]:
seq_len=63
pred_len=7
x_arr, y_arr,new_arr = transform_data(train_ft.values,mins,sizes,seq_len,pred_len)

In [113]:
x_arr.shape # batch,seq,feature

(839, 63, 18)

In [114]:
y_arr.shape

(839, 7)

In [115]:
class FinanceDataset(torch.utils.data.Dataset):
    def __init__(self,x,y=None):
        self.x = x
        self.y = y

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self,idx):
        item={}
        item['x'] = torch.Tensor(self.x[idx])
        if self.y is not None:
            item['y'] = torch.Tensor(self.y[idx])

        return item

In [116]:
dt = FinanceDataset(x_arr,y_arr)

In [117]:
dl = torch.utils.data.DataLoader(dt,batch_size=1,shuffle=False)
batch = next(iter(dl))
batch

{'x': tensor([[[0.4461, 0.4246, 0.4507,  ..., 0.0344, 0.0344, 0.2979],
          [0.4524, 0.4349, 0.4689,  ..., 0.0400, 0.0400, 0.2523],
          [0.4630, 0.4414, 0.4764,  ..., 0.0515, 0.0515, 0.3611],
          ...,
          [0.5132, 0.4971, 0.5231,  ..., 0.0603, 0.0603, 0.2044],
          [0.5187, 0.4989, 0.5148,  ..., 0.0396, 0.0396, 0.2703],
          [0.4915, 0.4772, 0.5078,  ..., 0.0415, 0.0415, 0.2144]]]),
 'y': tensor([[0.4891, 0.4954, 0.4997, 0.5110, 0.5114, 0.5111, 0.5045]])}

In [118]:
class moving_avg(torch.nn.Module):
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = torch.nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x

class series_decomp(torch.nn.Module):
  def __init__(self, kernel_size):
      super(series_decomp, self).__init__()
      self.moving_avg = moving_avg(kernel_size, stride=1)

  def forward(self, x):
      moving_mean = self.moving_avg(x)
      residual = x - moving_mean
      return moving_mean, residual

class DLinear(torch.nn.Module):
  def __init__(self, window_size, forcast_size, kernel_size, individual, feature_size):
      super().__init__()
      self.window_size = window_size
      self.forcast_size = forcast_size
      self.decompsition = series_decomp(kernel_size)
      self.individual = individual
      self.channels = feature_size
      self.fc_layer = torch.nn.Linear(feature_size, 1)
      if self.individual:
          self.Linear_Seasonal = torch.nn.ModuleList()
          self.Linear_Trend = torch.nn.ModuleList()
          for i in range(self.channels):
              self.Linear_Trend.append(torch.nn.Linear(self.window_size, self.forcast_size))
              self.Linear_Trend[i].weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))
              self.Linear_Seasonal.append(torch.nn.Linear(self.window_size, self.forcast_size))
              self.Linear_Seasonal[i].weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))
      else:
          self.Linear_Trend = torch.nn.Linear(self.window_size, self.forcast_size)
          self.Linear_Trend.weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))
          self.Linear_Seasonal = torch.nn.Linear(self.window_size,  self.forcast_size)
          self.Linear_Seasonal.weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))

  def forward(self, x):
      trend_init, seasonal_init = self.decompsition(x)
      trend_init, seasonal_init = trend_init.permute(0,2,1), seasonal_init.permute(0,2,1) # batch,feature,seq
      if self.individual:
          trend_output = torch.zeros([trend_init.size(0), trend_init.size(1), self.forcast_size], dtype=trend_init.dtype).to(trend_init.device)
          seasonal_output = torch.zeros([seasonal_init.size(0), seasonal_init.size(1), self.forcast_size], dtype=seasonal_init.dtype).to(seasonal_init.device)
          for idx in range(self.channels):
              trend_output[:, idx, :] = self.Linear_Trend[idx](trend_init[:, idx, :])
              seasonal_output[:, idx, :] = self.Linear_Seasonal[idx](seasonal_init[:, idx, :])
      else:
          trend_output = self.Linear_Trend(trend_init)
          seasonal_output = self.Linear_Seasonal(seasonal_init)
      x = seasonal_output + trend_output

      return self.fc_layer(x.permute(0,2,1)).squeeze()


In [119]:
model = DLinear(seq_len,pred_len,kernel_size=25,individual=False,feature_size=len(train_ft.columns))

In [120]:
tmp = model(batch['x'].to(device))
tmp

tensor([-0.3184, -0.2990, -0.3320, -0.2731, -0.3428, -0.3609, -0.4731],
       grad_fn=<SqueezeBackward0>)

In [121]:
def train_loop(dl,model,loss_fn,optimizer,device):

    epoch_loss = 0
    model.train()

    for batch in dl:
        pred = model(batch['x'].to(device))
        loss = loss_fn(pred,batch['y'].to(device))

        optimizer.zero_grad() # 경사 초기화
        loss.backward()# 역전파 부분
        optimizer.step() # 가중치 업데이트

        epoch_loss += loss.item()

    epoch_loss /= len(dl)

    return epoch_loss

In [122]:
@torch.inference_mode()
def test_loop(dl,model,loss_fn,device):

    epoch_loss = 0
    model.eval()
    pred_list = []
    for batch in dl:

        pred = model(batch['x'].to(device))

        if batch.get('y') is not None:
            loss = loss_fn(pred,batch['y'].to(device))
            epoch_loss += loss.item()
        pred = pred.to('cpu').numpy()
        pred_list.append(pred)

    pred = np.concatenate(pred_list) # 배치단위 예측값들 결합.
    epoch_loss /= len(dl)
    return epoch_loss,pred

In [123]:
class RMSELoss(torch.nn.Module):
    def __init__(self):
        super(RMSELoss,self).__init__()

    def forward(self,x,y):
        criterion = torch.nn.MSELoss()
        eps = 1e-6
        loss = torch.sqrt(criterion(x, y) + eps)
        return loss

In [124]:
n_splits = 5
window_size = seq_len
forcast_size = pred_len
kernel_size=25
batch_size = 32
individual = False
feature_size = len(train.columns)
epochs = 1000
loss_fn = RMSELoss()
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
cv = KFold(n_splits = n_splits,shuffle=True, random_state = SEED)

In [125]:
is_holdout = True
reset_seeds(SEED)
best_score_list = []
for i, (tri,vai) in enumerate(cv.split(x_arr)):
    model = DLinear(window_size,forcast_size,kernel_size,individual,feature_size).to(device)
    optimizer = torch.optim.Adam(model.parameters())

    # 학습용
    train_dt = FinanceDataset(x_arr[tri],y_arr[tri])
    train_dl = torch.utils.data.DataLoader(train_dt,batch_size=batch_size,shuffle=True)

    # 검증용
    valid_dt = FinanceDataset(x_arr[vai],y_arr[vai])
    valid_dl = torch.utils.data.DataLoader(valid_dt,batch_size=batch_size,shuffle=False)

    best_score = np.inf # 낮을수록 좋은 것이니까
    patience= 0
    for epoch in tqdm(range(epochs)):
        train_loss = train_loop(train_dl,model,loss_fn,optimizer,device)
        valid_loss,pred = test_loop(valid_dl,model,loss_fn,device)

        pred = pred*sizes[3]+mins[3]# 원상복귀를 해줘야함. 3번째(종가만 가져오면 됨.)
        y_true = y_arr[vai]*sizes[3]+mins[3]
        score = mean_squared_error(y_true,pred,squared=False)
        patience+=1

        if best_score > score :

            patience = 0
            print(score)
            best_score = score
            torch.save(model.state_dict(),f'{DATA_PATH}model7d_{i}.pth')

        if patience ==100 :
            break
    print(f'Fold[{i}] BEST RMSE : {best_score}')
    best_score_list.append(best_score)

    if is_holdout :
        break

# y_arr*sizes[3]+mins[3]

  0%|          | 0/1000 [00:00<?, ?it/s]

225.43885371997885
169.09337299645884
127.5001046648466
88.19462290646536
73.94083243624176
72.14440198140632
71.44988544923244
64.06686339420217
59.86365981791884
55.19336437521164
52.81362896814816
51.14661229202557
49.11776379757224
46.6643272965
46.26894434210707
44.83500396298442
44.02475299244217
43.88195610633836
Fold[0] BEST RMSE : 43.88195610633836


In [ ]:
batch['y']

tensor([[2033.7300, 2015.4399, 2022.6600, 2027.6100, 2040.4301, 2040.9399,
         2040.6000]])

In [ ]:
mov_avg = moving_avg(25,stride=1)

In [ ]:
mov_avg(X).shape

torch.Size([32, 3, 5])

In [ ]:
mov_avg = moving_avg(25,stride=1)
moving_mean = mov_avg(X)
print(X.size(), moving_mean.size())
res = X - moving_mean
print(res.size(), moving_mean.size())

torch.Size([32, 3, 5]) torch.Size([32, 3, 5])
torch.Size([32, 3, 5]) torch.Size([32, 3, 5])


In [ ]:
seasonal_init = res
trend_init = moving_mean

In [ ]:
seasonal_init, trend_init = seasonal_init.permute(0,2,1), trend_init.permute(0,2,1)

In [ ]:
seasonal_init.shape

torch.Size([32, 5, 3])

In [ ]:
season_layer = nn.Linear(in_seq_len, pred_len)

In [ ]:
trend_layer = nn.Linear(in_seq_len, pred_len)
seasonal_output = season_layer(seasonal_init)
trend_output =  trend_layer(trend_init)

In [ ]:
seasonal_output.shape

torch.Size([32, 5, 2])

In [ ]:
trend_output.shape

torch.Size([32, 5, 2])

In [ ]:
print(seasonal_output.size(),  trend_output.size(), (seasonal_output+trend_output).size())

torch.Size([32, 5, 2]) torch.Size([32, 5, 2]) torch.Size([32, 5, 2])


In [ ]:
(seasonal_output+trend_output).permute(0,2,1).size()

torch.Size([32, 2, 5])